# Projeto 1 - Ciência dos Dados

Nome: Adney Costa Moura

Nome: Lister Ogusuku Ribeiro

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\adney\OneDrive\Documentos\2° semestre - Insper\Ciência dos Dados\ProjetoCdados_AdneyLister


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
filename = 'nubank.xlsx'

In [39]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Relevância
0,@nubank você me ilude tão bem 🤩🤩🤩,0
1,"vou ter que fazer parcelamento da parcela, eu ...",0
2,@hs_ramao @nubank aumentaram o meu e eu nem us...,1
3,hoje eu mal acordei e tava pegando ônibus o re...,0
4,🤣🤣🤣 @nubank vamos amigar https://t.co/7k6syhkxf0,0


In [5]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Relevância
0,"@ogustamlk gu, obrigado por nos avisar sobre i...",0
1,@nubank nubank libera logo meu cartao de crédi...,1
2,nubank foi o melhor cartão que eu já fiz até h...,1
3,@nubank quer casar cmg?,0
4,"aeeee carai, nubank aceitou meu crédito :3",1


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

Nosso produto foi a startup de serviços financeiros Nubank. Consideramos como relevante as operações que envolvem o limite de crédito, interação com o cliente e reclamações.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [6]:
# https://docs.python.org/3/library/re.html#
import re 


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

In [7]:
# Separando cada row do data frame, fazendo um split das palavras e limpando cada frase. Além disso foi filtrado cada row
# as listas de relevância.
lista_relevante = []
lista_irrelevante = []

for i in range(0,500):
    # Separando os rows do DataFrame
    x = cleanup(train.Treinamento[i]).lower().split()
 
    if train.iloc[i,1] == 1:
        j = 0
        while j < len(x):
            lista_relevante.append(x[j])
            j+=1
        
    else:
        k = 0
        while k < len(x):
            lista_irrelevante.append(x[k])
            k+=1

# Tweets Relevantes

In [8]:
# Guardando as palavras como um pd.Series
serie_relevante = pd.Series(lista_relevante)
serie_relevante.head()

0     @hs_ramao
1       @nubank
2    aumentaram
3             o
4           meu
dtype: object

##### Frequências Absolutas

In [9]:
tabela_relevante_absoluta = serie_relevante.value_counts()
tabela_relevante_absoluta

@nubank         135
o               117
nubank           97
de               95
meu              94
               ... 
ligo              1
expor             1
tao               1
@matheusdssx      1
ficaria           1
Length: 1091, dtype: int64

##### Frequências Relativas

In [10]:
tabela_relevante_relativa = serie_relevante.value_counts(True)
tabela_relevante_relativa

@nubank         0.038516
o               0.033381
nubank          0.027675
de              0.027104
meu             0.026819
                  ...   
ligo            0.000285
expor           0.000285
tao             0.000285
@matheusdssx    0.000285
ficaria         0.000285
Length: 1091, dtype: float64

# Tweets Irrelevantes

In [11]:
# Guardando as palavras como um pd.Series
serie_irrelevante = pd.Series(lista_irrelevante)
serie_irrelevante.head()

0    @nubank
1       você
2         me
3      ilude
4        tão
dtype: object

##### Frequências Absolutas

In [12]:
tabela_irrelevante_absoluta = serie_irrelevante.value_counts()
tabela_irrelevante_absoluta

de          132
@nubank     125
o           116
que         109
a           106
           ... 
guarana       1
serve         1
amg           1
previsão      1
25            1
Length: 1780, dtype: int64

##### Frequências Relativas

In [13]:
tabela_irrelevante_relativa = serie_irrelevante.value_counts(True)
tabela_irrelevante_relativa

de          0.025414
@nubank     0.024066
o           0.022333
que         0.020986
a           0.020408
              ...   
guarana     0.000193
serve       0.000193
amg         0.000193
previsão    0.000193
25          0.000193
Length: 1780, dtype: float64

# Tweets Totais

In [14]:
total = lista_relevante + lista_irrelevante

In [15]:
total_series = pd.Series(total)

##### Frequências Absolutas

In [16]:
tabela_total = total_series.value_counts()
tabela_total

@nubank       260
o             233
de            227
nubank        191
que           166
             ... 
conectado       1
rastreador      1
venham          1
resolvi         1
25              1
Length: 2392, dtype: int64

##### Frequências Relativas

In [17]:
tabela_total_relativa = total_series.value_counts(True)
tabela_total_relativa

@nubank       0.029888
o             0.026785
de            0.026095
nubank        0.021957
que           0.019083
                ...   
conectado     0.000115
rastreador    0.000115
venham        0.000115
resolvi       0.000115
25            0.000115
Length: 2392, dtype: float64

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [18]:
lista_test = []
lista_relevante_test = []
lista_irrelevante_test = []

for i in range(0,300):
    x = cleanup(test.Teste[i]).lower().split()
    
    lista_test.append(x)
    
    if train.iloc[i,1] == 1:
        j = 0
        while j < len(x):
            lista_relevante_test.append(x[j])
            j+=1
        
    else:
        k = 0
        while k < len(x):
            lista_irrelevante_test.append(x[k])
            k+=1

In [26]:
test["Relevância_Classificador"] = 'a'
test.head()

,Teste,Relevância,Relevância_Classificador
0,"@ogustamlk gu, obrigado por nos avisar sobre i...",0,a
1,@nubank nubank libera logo meu cartao de crédi...,1,a
2,nubank foi o melhor cartão que eu já fiz até h...,1,a
3,@nubank quer casar cmg?,0,a
4,"aeeee carai, nubank aceitou meu crédito :3",1,a


In [42]:
contador = 0
Pr = len(lista_relevante)/len(total)
Pi = len(lista_irrelevante)/len(total)
Prob_d_ser_relevante = []
Prob_d_ser_irrelevante = []

while contador < len(lista_test):
    
    acumulador_relevante = 1
    acumulador_irrelevante = 1

    for h in lista_test[contador]:

        if h not in lista_relevante:
            tabela_relevante_absoluta[h] = 0

        if h not in lista_irrelevante:
            tabela_irrelevante_absoluta[h] = 0

        acumulador_relevante *= ( tabela_relevante_absoluta[h] + 1 ) / ( len(lista_relevante) + len(total) )
        acumulador_irrelevante *= ( tabela_irrelevante_absoluta[h] + 1 ) / ( len(lista_irrelevante) + len(total) ) 
      
        
    # Probabilidades
    Prob_d_ser_relevante.append(Pr*acumulador_relevante)
    Prob_d_ser_irrelevante.append(Pi*acumulador_irrelevante)

    if Prob_d_ser_relevante[contador] > Prob_d_ser_irrelevante[contador]:
        test.loc[contador,["Relevância_Classificador"]] = 1

    else:
        test.loc[contador,["Relevância_Classificador"]] = 0
        
    contador = contador + 1

In [43]:
test.head()

,Teste,Relevância,Relevância_Classificador
0,"@ogustamlk gu, obrigado por nos avisar sobre i...",0,0
1,@nubank nubank libera logo meu cartao de crédi...,1,1
2,nubank foi o melhor cartão que eu já fiz até h...,1,0
3,@nubank quer casar cmg?,0,1
4,"aeeee carai, nubank aceitou meu crédito :3",1,1


##### Analisando o Classificador

In [30]:
Verdadeiros_Positivos = test.loc[(test["Relevância"] == 1) & (test["Relevância_Classificador"] == 1), :]
Falsos_Positivos = test.loc[(test["Relevância"] == 0) & (test["Relevância_Classificador"] == 1), :]
Verdadeiros_Negativos = test.loc[(test["Relevância"] == 0) & (test["Relevância_Classificador"] == 0), :]
Falsos_Negativos = test.loc[(test["Relevância"] == 1) & (test["Relevância_Classificador"] == 0), :]

In [38]:
pd.crosstab(test["Relevância"], test["Relevância_Classificador"], normalize=True)

Relevância_Classificador,0,1
Relevância,,
0,0.400000,0.193333
1,0.116667,0.290000


___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)


___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**